<a href="https://colab.research.google.com/github/eleazarpsmr/notebook-colab/blob/main/Model_EPS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install openpyxl

In [3]:
from google.colab import files
import pandas as pd

# Upload file
uploaded = files.upload()

# Baca file Excel
df = pd.read_excel('XAUUSD_1m_2024.xlsx', engine='openpyxl')  # Ganti nama file sesuai yang Anda upload
trades = pd.read_excel('trade_history_2024.xlsx', engine='openpyxl')

Saving trade_history_2024.xlsx to trade_history_2024.xlsx
Saving XAUUSD_1m_2024.xlsx to XAUUSD_1m_2024.xlsx


In [9]:
# Cek 5 baris pertama data harga
print(df.head())

# Cek 5 baris pertama riwayat trade
print(trades.head())

            Timestamp     Open     High      Low    Close  Volume
0 2024-01-02 01:00:00  2062.91  2064.79  2062.80  2064.02      62
1 2024-01-02 01:01:00  2063.85  2064.46  2063.79  2064.45      39
2 2024-01-02 01:02:00  2064.45  2064.50  2064.24  2064.28      33
3 2024-01-02 01:03:00  2064.30  2064.55  2064.28  2064.30      35
4 2024-01-02 01:04:00  2064.30  2065.51  2064.24  2065.37      53
         Entry Time         Exit Time Position (Buy/Sell)  Entry Price  \
0  2024.01.04 17:59  2024.01.04 19:03                 Buy      2045.05   
1  2024.01.05 17:24  2024.01.05 17:30                 Buy      2062.22   
2  2024.01.12 17:37  2024.01.12 17:43                Sell      2054.59   
3  2024.01.17 17:16  2024.01.17 17:31                Sell      2011.00   
4  2024.01.18 16:17  2024.01.18 16:21                Sell      2010.29   

   Exit Price  Profit/Loss  
0     2043.40      -386.10  
1     2058.56      -376.98  
2     2052.07       425.88  
3     2007.94       428.40  
4     2012.13 

In [10]:
print(df.columns)


Index(['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume'], dtype='object')


In [12]:
# Cek apakah ada data yang hilang
print("Missing Values:\n", df.isnull().sum())

# Cek apakah ada baris duplikat
print(f"Jumlah baris duplikat: {df.duplicated().sum()}")


Missing Values:
 Timestamp    0
Open         0
High         0
Low          0
Close        0
Volume       0
dtype: int64
Jumlah baris duplikat: 0


In [13]:
# Ubah ke format datetime
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

# Cek apakah timestamp urut menaik
print("Sudah urut waktu:", df['Timestamp'].is_monotonic_increasing)


Sudah urut waktu: True


In [14]:
# Hitung selisih antar baris dalam detik
df['delta'] = df['Timestamp'].diff().dt.total_seconds()

# Lihat distribusi interval waktu
print("Distribusi selisih waktu (detik):\n", df['delta'].value_counts().head())

# Hapus kolom bantu
df.drop(columns='delta', inplace=True)


Distribusi selisih waktu (detik):
 delta
60.0        355393
3720.0         188
176640.0        46
120.0           16
3660.0          10
Name: count, dtype: int64


In [15]:
# Pastikan kolom waktu pada trades dalam format datetime
trades['Entry Time'] = pd.to_datetime(trades['Entry Time'])
trades['Exit Time'] = pd.to_datetime(trades['Exit Time'])

# Cek hasilnya
print(trades.dtypes)


Entry Time             datetime64[ns]
Exit Time              datetime64[ns]
Position (Buy/Sell)            object
Entry Price                   float64
Exit Price                    float64
Profit/Loss                   float64
dtype: object


In [16]:
# Untuk mempercepat lookup, ubah df menjadi indexed by Timestamp
df_indexed = df.set_index('Timestamp')

# Fungsi bantu untuk mengambil baris harga saat entry dan exit
def get_price_at_time(entry_time, exit_time):
    entry_row = df_indexed.loc[:entry_time].iloc[-1]
    exit_row = df_indexed.loc[:exit_time].iloc[-1]
    return pd.Series({
        'Open_Entry': entry_row['Open'],
        'High_Entry': entry_row['High'],
        'Low_Entry': entry_row['Low'],
        'Close_Entry': entry_row['Close'],
        'Volume_Entry': entry_row['Volume'],
        'Open_Exit': exit_row['Open'],
        'High_Exit': exit_row['High'],
        'Low_Exit': exit_row['Low'],
        'Close_Exit': exit_row['Close'],
        'Volume_Exit': exit_row['Volume'],
    })

# Terapkan ke semua baris trades
trade_details = trades.apply(lambda row: get_price_at_time(row['Entry Time'], row['Exit Time']), axis=1)

# Gabungkan dengan dataframe trades
trades_augmented = pd.concat([trades, trade_details], axis=1)


In [17]:
# Lihat hasil sinkronisasi
print(trades_augmented.head())


           Entry Time           Exit Time Position (Buy/Sell)  Entry Price  \
0 2024-01-04 17:59:00 2024-01-04 19:03:00                 Buy      2045.05   
1 2024-01-05 17:24:00 2024-01-05 17:30:00                 Buy      2062.22   
2 2024-01-12 17:37:00 2024-01-12 17:43:00                Sell      2054.59   
3 2024-01-17 17:16:00 2024-01-17 17:31:00                Sell      2011.00   
4 2024-01-18 16:17:00 2024-01-18 16:21:00                Sell      2010.29   

   Exit Price  Profit/Loss  Open_Entry  High_Entry  Low_Entry  Close_Entry  \
0     2043.40      -386.10     2045.00     2045.16    2044.55      2045.08   
1     2058.56      -376.98     2062.17     2064.03    2062.16      2063.11   
2     2052.07       425.88     2054.59     2054.98    2053.98      2054.18   
3     2007.94       428.40     2011.00     2011.34    2009.84      2010.35   
4     2012.13      -386.40     2010.29     2010.49    2009.57      2009.91   

   Volume_Entry  Open_Exit  High_Exit  Low_Exit  Close_Exit  V

In [21]:
!pip install ta


  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=61d20ee536117045bb88658ab899d8919c4786b7a3a5a03e439d543fc59c91c5
  Stored in directory: /root/.cache/pip/wheels/a1/d7/29/7781cc5eb9a3659d032d7d15bdd0f49d07d2b24fec29f44bc4
Successfully built ta


In [22]:
import pandas as pd
import numpy as np
import ta
import warnings
warnings.filterwarnings('ignore')

# Pastikan Timestamp sebagai datetime
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

# Urutkan dan reset index
df = df.sort_values('Timestamp').reset_index(drop=True)

# =============================
# 1. Moving Averages (LWMA & HMA)
# =============================

def lwma(series, window):
    weights = np.arange(1, window + 1)
    return series.rolling(window).apply(lambda x: np.dot(x, weights)/weights.sum(), raw=True)

def hma(series, window):
    half = int(window / 2)
    sqrt = int(np.sqrt(window))
    wma_half = lwma(series, half)
    wma_full = lwma(series, window)
    raw_hma = 2 * wma_half - wma_full
    return lwma(raw_hma, sqrt)

# Tambahkan LWMA dan HMA yang digunakan
df['LWMA10'] = lwma(df['Close'], 10)
df['LWMA13'] = lwma(df['Close'], 13)
df['LWMA20'] = lwma(df['Close'], 20)
df['LWMA80'] = lwma(df['Close'], 80)
df['LWMA100'] = lwma(df['Close'], 100)
df['LWMA200'] = lwma(df['Close'], 200)
df['HMA10'] = hma(df['Close'], 10)
df['HMA13'] = hma(df['Close'], 13)

# =============================
# 2. Bollinger Band + BBWidth
# =============================
bb_indicator = ta.volatility.BollingerBands(close=df['Close'], window=20, window_dev=2)
df['Upper_Band'] = bb_indicator.bollinger_hband()
df['Lower_Band'] = bb_indicator.bollinger_lband()

df['BBWidth_1'] = df['Upper_Band'] - df['Lower_Band']
df['emaBBWidth_1'] = df['BBWidth_1'].ewm(span=10, adjust=False).mean()
df['bbWidthCondition_wide'] = df['BBWidth_1'] > df['emaBBWidth_1']

# =============================
# 3. Stochastic Oscillator
# =============================
stoch = ta.momentum.StochasticOscillator(high=df['High'], low=df['Low'], close=df['Close'], window=8, smooth_window=3)
df['Stoch_K'] = stoch.stoch()
df['Stoch_D'] = stoch.stoch_signal()
df['stoch_avg'] = (df['Stoch_K'] + df['Stoch_D']) / 2

# =============================
# 4. MACD Histogram & Main
# =============================
macd = ta.trend.MACD(close=df['Close'])
df['MACDMain'] = macd.macd()
df['MACDHist'] = macd.macd_diff()

# =============================
# 5. ADX (1 menit, window=10)
# =============================
adx = ta.trend.ADXIndicator(high=df['High'], low=df['Low'], close=df['Close'], window=10)
df['ADX'] = adx.adx()

# =============================
# 6. ADXHTF (60 menit, window=7)
# =============================
df_60m = df.set_index('Timestamp').resample('60T').agg({
    'Open': 'first',
    'High': 'max',
    'Low': 'min',
    'Close': 'last'
}).dropna().reset_index()

adx_60m = ta.trend.ADXIndicator(
    high=df_60m['High'],
    low=df_60m['Low'],
    close=df_60m['Close'],
    window=7
)
df_60m['ADXHTF'] = adx_60m.adx()
df_60m = df_60m[['Timestamp', 'ADXHTF']]

# Merge ke df utama (1-menit) dengan ffill
df = df.merge(df_60m, on='Timestamp', how='left')
df['ADXHTF'] = df['ADXHTF'].ffill()


In [23]:
# Resample 1-menit ke 60-menit, lalu hitung ADXHTF
df_60m = df.set_index('Timestamp').resample('60T').agg({
    'High': 'max',
    'Low': 'min',
    'Close': 'last'
}).dropna()

adx_htf = ta.trend.ADXIndicator(high=df_60m['High'], low=df_60m['Low'], close=df_60m['Close'], window=7)
df_60m['ADXHTF'] = adx_htf.adx()

# Gabungkan kembali ke data 1-menit (merge asof)
df = df.sort_values('Timestamp')
df_60m = df_60m[['ADXHTF']].reset_index().sort_values('Timestamp')

df = pd.merge_asof(df, df_60m, on='Timestamp', direction='backward')


In [26]:
print(df.columns.tolist())


['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume', 'LWMA10', 'LWMA13', 'LWMA20', 'LWMA80', 'LWMA100', 'LWMA200', 'HMA10', 'HMA13', 'Upper_Band', 'Lower_Band', 'BBWidth_1', 'emaBBWidth_1', 'bbWidthCondition_wide', 'Stoch_K', 'Stoch_D', 'stoch_avg', 'MACDMain', 'MACDHist', 'ADX', 'ADXHTF_x', 'ADXHTF_y']


In [27]:
# Langkah 4.1 - Revisi: Definisikan fitur yang digunakan dalam strategi
feature_cols = [
    'Open', 'High', 'Low', 'Close', 'Volume',
    'LWMA10', 'LWMA13', 'LWMA20', 'LWMA80', 'LWMA100', 'LWMA200',
    'HMA10', 'HMA13',
    'Upper_Band', 'Lower_Band',
    'BBWidth_1', 'emaBBWidth_1', 'bbWidthCondition_wide',
    'Stoch_K', 'Stoch_D', 'stoch_avg',
    'MACDMain', 'MACDHist',
    'ADX', 'ADXHTF_x', 'ADXHTF_y'
]

# Pastikan hanya ambil kolom yang tersedia
available_cols = [col for col in feature_cols if col in df.columns]

# Subset dataframe untuk analisis
df_features = df[available_cols].copy()
df_features.head()


,Open,High,Low,Close,Volume,LWMA10,LWMA13,LWMA20,LWMA80,LWMA100,...,emaBBWidth_1,bbWidthCondition_wide,Stoch_K,Stoch_D,stoch_avg,MACDMain,MACDHist,ADX,ADXHTF_x,ADXHTF_y
0,2062.91,2064.79,2062.80,2064.02,62,NaN,NaN,NaN,NaN,NaN,...,NaN,False,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
1,2063.85,2064.46,2063.79,2064.45,39,NaN,NaN,NaN,NaN,NaN,...,NaN,False,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
2,2064.45,2064.50,2064.24,2064.28,33,NaN,NaN,NaN,NaN,NaN,...,NaN,False,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
3,2064.30,2064.55,2064.28,2064.30,35,NaN,NaN,NaN,NaN,NaN,...,NaN,False,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
4,2064.30,2065.51,2064.24,2065.37,53,NaN,NaN,NaN,NaN,NaN,...,NaN,False,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0


In [29]:
# Load ulang trade log
df_trades = pd.read_excel('trade_history_2024.xlsx')

# Pastikan kolom waktu dalam format datetime
df_trades['Entry Time'] = pd.to_datetime(df_trades['Entry Time'])
df_trades['Exit Time'] = pd.to_datetime(df_trades['Exit Time'])

# Tampilkan beberapa baris pertama
df_trades.head()


,Entry Time,Exit Time,Position (Buy/Sell),Entry Price,Exit Price,Profit/Loss
0,2024-01-04 17:59:00,2024-01-04 19:03:00,Buy,2045.05,2043.40,-386.10
1,2024-01-05 17:24:00,2024-01-05 17:30:00,Buy,2062.22,2058.56,-376.98
2,2024-01-12 17:37:00,2024-01-12 17:43:00,Sell,2054.59,2052.07,425.88
3,2024-01-17 17:16:00,2024-01-17 17:31:00,Sell,2011.00,2007.94,428.40
4,2024-01-18 16:17:00,2024-01-18 16:21:00,Sell,2010.29,2012.13,-386.40


In [30]:
# Salin df untuk modifikasi
df_labeled = df.copy()
df_labeled['Market_Condition'] = np.nan  # Kolom baru untuk label

# Loop setiap baris di trade log dan tandai bar entry
for _, trade in df_trades.iterrows():
    entry_time = pd.to_datetime(trade['Entry Time'])
    pnl = trade['Profit/Loss']

    # Temukan bar dengan timestamp == entry_time
    idx = df_labeled[df_labeled['Timestamp'] == entry_time].index
    if not idx.empty:
        df_labeled.at[idx[0], 'Market_Condition'] = 'Favorable' if pnl >= 0 else 'Unfavorable'

# Cek hasil label
df_labeled['Market_Condition'].value_counts(dropna=False)


,count
Market_Condition,
NaN,355458
Favorable,116
Unfavorable,99


In [31]:
# Ambil subset data yang berlabel
df_labeled_subset = df_labeled[df_labeled['Market_Condition'].notna()]

# Hitung statistik deskriptif per kondisi
summary_stats = df_labeled_subset.groupby('Market_Condition').agg(['mean', 'std'])

# Tampilkan ringkasan
summary_stats.T  # Transpose supaya lebih mudah dibaca


Market_Condition                                Favorable  \
Timestamp             mean  2024-07-10 17:46:36.724137728   
                      std      95 days 05:43:46.145113603   
Open                  mean                    2400.804052   
                      std                      198.366488   
High                  mean                    2401.403017   
                      std                      198.375165   
Low                   mean                    2400.099052   
                      std                      198.417816   
Close                 mean                    2400.770259   
                      std                       198.44946   
Volume                mean                     210.241379   
                      std                      254.896965   
LWMA10                mean                    2400.812661   
                      std                      198.325042   
LWMA13                mean                    2400.830029   
                      std                      198.323837   
LWMA20                mean                    2400.866144   
                      std                      198.324355   
LWMA80                mean                    2400.853977   
                      std                      198.302948   
LWMA100               mean                    2400.780328   
                      std                       198.33229   
LWMA200               mean                     2400.75542   
                      std                      198.749485   
HMA10                 mean                    2400.789118   
                      std                      198.313692   
HMA13                 mean                    2400.772621   
                      std                      198.308053   
Upper_Band            mean                    2404.186539   
                      std                      198.472875   
Lower_Band            mean                    2397.579935   
                      std                      198.210119   
BBWidth_1             mean                       6.606604   
                      std                        3.467644   
emaBBWidth_1          mean                       6.018772   
                      std                        2.884544   
bbWidthCondition_wide mean                       0.689655   
                      std                        0.464642   
Stoch_K               mean                      52.822705   
                      std                       38.215362   
Stoch_D               mean                      52.465407   
                      std                       37.523393   
stoch_avg             mean                      52.644056   
                      std                       37.551509   
MACDMain              mean                       -0.03689   
                      std                        1.022295   
MACDHist              mean                      -0.019973   
                      std                         0.42053   
ADX                   mean                      28.277319   
                      std                        9.744591   
ADXHTF_x              mean                      33.756053   
                      std                       11.982211   
ADXHTF_y              mean                      33.756053   
                      std                       11.982211   

Market_Condition                              Unfavorable  
Timestamp             mean  2024-07-05 18:43:09.090908928  
                      std     103 days 08:46:23.408636877  
Open                  mean                    2390.130101  
                      std                      225.988544  
High                  mean                    2390.818586  
                      std                       226.02361  
Low                   mean                    2389.540808  
                      std                      225.969933  
Close                 mean                    2390.227677  
                      std                   

In [35]:
import pandas as pd

# 1. Load data
price_data = pd.read_excel("/content/XAUUSD_1m_2024.xlsx")
trade_history = pd.read_excel("/content/trade_history_2024.xlsx")

# 2. Pastikan kolom waktu dalam datetime
price_data['Timestamp'] = pd.to_datetime(price_data['Timestamp'])
trade_history['Entry Time'] = pd.to_datetime(trade_history['Entry Time'])
trade_history['Exit Time'] = pd.to_datetime(trade_history['Exit Time'])

# 3. Untuk setiap baris trade, tambahkan fitur dari price_data di Entry Time
def get_price_features(row):
    entry_row = price_data[price_data['Timestamp'] == row['Entry Time']]
    exit_row = price_data[price_data['Timestamp'] == row['Exit Time']]

    # Jika waktu tidak cocok persis, cari baris terdekat
    if entry_row.empty:
        entry_row = price_data.iloc[(price_data['Timestamp'] - row['Entry Time']).abs().argsort()[:1]]
    if exit_row.empty:
        exit_row = price_data.iloc[(price_data['Timestamp'] - row['Exit Time']).abs().argsort()[:1]]

    entry_data = entry_row.iloc[0]
    exit_data = exit_row.iloc[0]

    return pd.Series({
        'Entry_Close': entry_data['Close'],
        'Entry_Volume': entry_data['Volume'],
        'Exit_Close': exit_data['Close'],
        'Exit_Volume': exit_data['Volume']
    })

# 4. Gabungkan data fitur ke trade_history
features = trade_history.apply(get_price_features, axis=1)
df_merged = pd.concat([trade_history, features], axis=1)

# 5. Pastikan Profit/Loss numerik
df_merged['Profit/Loss'] = pd.to_numeric(df_merged['Profit/Loss'], errors='coerce')


In [38]:
print(df_merged.columns)


Index(['Entry Time', 'Exit Time', 'Position (Buy/Sell)', 'Entry Price',
       'Exit Price', 'Profit/Loss', 'Entry_Close', 'Entry_Volume',
       'Exit_Close', 'Exit_Volume'],
      dtype='object')


In [39]:
df_merged.columns = df_merged.columns.str.strip()


In [40]:
print(df_merged.columns)


Index(['Entry Time', 'Exit Time', 'Position (Buy/Sell)', 'Entry Price',
       'Exit Price', 'Profit/Loss', 'Entry_Close', 'Entry_Volume',
       'Exit_Close', 'Exit_Volume'],
      dtype='object')


In [41]:
print(df_trades.columns)


Index(['Entry Time', 'Exit Time', 'Position (Buy/Sell)', 'Entry Price',
       'Exit Price', 'Profit/Loss'],
      dtype='object')


In [42]:
df_merged.head()


,Entry Time,Exit Time,Position (Buy/Sell),Entry Price,Exit Price,Profit/Loss,Entry_Close,Entry_Volume,Exit_Close,Exit_Volume
0,2024-01-04 17:59:00,2024-01-04 19:03:00,Buy,2045.05,2043.40,-386.10,2045.08,416.0,2043.69,326.0
1,2024-01-05 17:24:00,2024-01-05 17:30:00,Buy,2062.22,2058.56,-376.98,2063.11,1407.0,2058.85,1319.0
2,2024-01-12 17:37:00,2024-01-12 17:43:00,Sell,2054.59,2052.07,425.88,2054.18,1200.0,2051.72,1034.0
3,2024-01-17 17:16:00,2024-01-17 17:31:00,Sell,2011.00,2007.94,428.40,2010.35,1079.0,2007.72,1114.0
4,2024-01-18 16:17:00,2024-01-18 16:21:00,Sell,2010.29,2012.13,-386.40,2009.91,793.0,2012.10,647.0


In [43]:
# Pastikan kolom Profit/Loss numerik
df_merged['Profit/Loss'] = pd.to_numeric(df_merged['Profit/Loss'], errors='coerce')

# Pisahkan berdasarkan posisi dan profitabilitas
df_buy_fav = df_merged[(df_merged['Position (Buy/Sell)'] == 'Buy') & (df_merged['Profit/Loss'] > 0)]
df_sell_fav = df_merged[(df_merged['Position (Buy/Sell)'] == 'Sell') & (df_merged['Profit/Loss'] > 0)]
df_buy_unfav = df_merged[(df_merged['Position (Buy/Sell)'] == 'Buy') & (df_merged['Profit/Loss'] < 0)]
df_sell_unfav = df_merged[(df_merged['Position (Buy/Sell)'] == 'Sell') & (df_merged['Profit/Loss'] < 0)]


In [45]:
indikator_cols = ['Entry_Close', 'LWMA20', 'LWMA100', 'HMA10', 'HMA13',
                  'ADX', 'ADXHTF', 'stoch', 'MACD_Line', 'Signal_Line']

def summary_stats(df, label):
    print(f"\n{label}")
    display(df[indikator_cols].describe())

summary_stats(df_buy_fav, "Buy Favorable (Profit)")
summary_stats(df_buy_unfav, "Buy Unfavorable (Loss)")
summary_stats(df_sell_fav, "Sell Favorable (Profit)")
summary_stats(df_sell_unfav, "Sell Unfavorable (Loss)")



Buy Favorable (Profit)


KeyError: "['LWMA20', 'LWMA100', 'HMA10', 'HMA13', 'ADX', 'ADXHTF', 'stoch', 'MACD_Line', 'Signal_Line'] not in index"